In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Restrict minor warnings
import warnings
warnings.filterwarnings('ignore')

# to display all outputs of one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = 100

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer as CTT
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader

import syft as sy
from uuid import UUID
from uuid import uuid4


In [3]:
from src.psi.util import Client, Server
from src.utils import add_ids
from src.utils.data_utils import id_collate_fn

In [63]:
def get_classes_weights(class1_size, class2_size):
    if class1_size < class2_size:
        return [class2_size / class1_size, 1]
        factor2 = 1
    else:
        return [1, class1_size / class2_size]


## Create datasets

In [51]:
fn = "/ssd003/projects/pets/datasets/caravan-insurance-challenge.csv"
df1 = pd.read_csv(fn)

In [52]:
df1.shape

(9822, 87)

In [53]:
categorical_cols = ['MOSTYPE','MOSHOOFD']
df = pd.get_dummies(df1, columns = categorical_cols)


In [54]:
train = df[df['ORIGIN']=='train']
val = df[df['ORIGIN']=='test']

_ = train.pop('ORIGIN')
_ = val.pop('ORIGIN')

X_train = train
X_val = val
y_train = train.pop('CARAVAN')
y_val = val.pop('CARAVAN')

In [55]:
X_train.shape, y_train.shape

((5822, 133), (5822,))

In [56]:
X_val.shape, y_val.shape

((4000, 133), (4000,))

In [58]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)


In [ ]:
scaler = StandardScaler() # change?
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [59]:
class dataset(Dataset):
    
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]
    
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return self.length


In [73]:
dataset_train = dataset(X_train, y_train)
dataset_val = dataset(X_val, y_val)


In [74]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [75]:
# DataLoader
# tuning batch_size
dataloader_train = DataLoader(dataset_train, batch_size=512, shuffle=True)
dataload_val = DataLoader(dataset_val, batch_size=512, shuffle=False)

## Training

In [64]:
# check relative weights of classes
class1_size = df[df['CARAVAN'] == 1].shape[0]
class0_size = df[df['CARAVAN'] == 0].shape[0]
print(class1_size, class0_size)
weights = get_classes_weights(class1_size, class0_size)
print(weights)
weights = torch.tensor(weights).to(device)
#criterion = nn.?(weight=weights)

586 9236
[15.761092150170649, 1]


In [70]:
# Model , Optimizer, Loss
model = torch.nn.Sequential(
            nn.Linear(X_train.shape[1], 40),
            nn.ReLU(), #
            nn.Linear(40, 40),
            nn.ReLU(), #
            nn.Linear(40, 40),
            nn.Linear(40, 40),
            nn.Linear(40, 1),
            nn.Sigmoid()
        )

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

#optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
#loss_fn = nn.BCELoss()

In [31]:
# device to train on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [71]:
model.to(device)

Sequential(
  (0): Linear(in_features=133, out_features=40, bias=True)
  (1): Linear(in_features=40, out_features=40, bias=True)
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): Linear(in_features=40, out_features=40, bias=True)
  (4): Linear(in_features=40, out_features=1, bias=True)
  (5): Sigmoid()
)

In [68]:
#forward loop
learning_rate = 0.01
epochs = 500

losses = []
accur = []
for i in range(epochs):
    for j,(x_train, y_train) in enumerate(dataloader_train):
        # calculate output
        output = model(x_train)

        # calculate loss
        loss = loss_fn(output, y_train.reshape(-1, 1))

        # accuracy
        predicted = model(torch.tensor(output, dtype=torch.float32))
        acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i%50 == 0:
        losses.append(loss)
        accur.append(acc)
        print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))


NameError: name 'x' is not defined

In [ ]:
# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")


In [ ]:
# f1 instead of auc on validation set